In [ ]:
import uuid
import os
#import importlib
from utils import ComfyUIClient

# Configuration du serveur ComfyUI 
server_address = "127.0.0.1:8188" # faire attention sur Runpod address
client_id = str(uuid.uuid4())

#client ComfyUI 
comfyui_client = ComfyUIClient(server_address=server_address, client_id=client_id)

In [ ]:
# Test du pipeline avec plusieurs valeurs de CFG
comfyui_client.run_pipeline(
    input_dir="input_test",
    output_dir="final_outputs",
    workflow_json_path="sdx_turbo_input_images.json",
    n_images=3,
    positive_prompt="Blue fruit with a samurai helmet, high detail, intricate, sharp focus, artstation, trending on artstation, 8k",
    denoise=0.9,
    cfg_list=[1]  # Teste 3 valeurs de CFG différentes
)

In [ ]:
from PIL import Image, ImageDraw, ImageFilter
import numpy as np

def make_center_soft_rect_mask(
    img_w: int,
    img_h: int,
    fill_ratio: float = 0.75,        # rectangle size = fill_ratio * (W,H)
    max_value: int = 255,            # reduce strength vs 255
    feather_px: int = 60,            # soft decay (Gaussian blur radius)
    edge_band_px: int = 120,         # where we add “random border” (in/out band around edge)
    edge_noise_amp: float = 0.35,    # 0..1, how strong the randomness is on the edge band
    out_path: str = "mask.png",
    seed: int | None = None
) -> Image.Image:
    """
    Centered rectangular mask with soft decay + randomized borders (no hard straight line).
    Returns PIL mask (mode "L") and saves it to out_path.
    """

    # clamp / sanitize
    fill_ratio = float(max(0.0, min(1.0, fill_ratio)))
    max_value = int(max(0, min(255, max_value)))
    feather_px = int(max(0, feather_px))
    edge_band_px = int(max(0, edge_band_px))
    edge_noise_amp = float(max(0.0, min(1.0, edge_noise_amp)))

    rng = np.random.default_rng(seed)

    # 1) base rectangle (hard) at reduced intensity
    mask = Image.new("L", (img_w, img_h), 0)
    draw = ImageDraw.Draw(mask)

    rect_w = int(round(img_w * fill_ratio))
    rect_h = int(round(img_h * fill_ratio))

    x0 = (img_w - rect_w) // 2 
    y0 = (img_h - rect_h) // 2
    x1 = x0 + rect_w
    y1 = y0 + rect_h

    draw.rectangle([x0, y0, x1, y1], fill=max_value)

    # 2) soft decay on edges
    if feather_px > 0:
        mask = mask.filter(ImageFilter.GaussianBlur(radius=feather_px))

    # 3) add randomness only near the border (band), to avoid straight “cut” lines
    if edge_band_px > 0 and edge_noise_amp > 0.0:
        m = np.asarray(mask, dtype=np.float32) / 255.0  # 0..1

        # approximate "edge band": pixels that are not deep inside or fully outside
        # (thresholds chosen to isolate the transition region after blur)
        band = (m > 0.05) & (m < 0.95)

        # make the band thicker if needed by dilating it in a cheap way:
        # blur the band mask then threshold it
        band_img = Image.fromarray((band.astype(np.uint8) * 255), mode="L")
        if edge_band_px > 0:
            band_img = band_img.filter(ImageFilter.GaussianBlur(radius=edge_band_px / 4))
        band = (np.asarray(band_img, dtype=np.float32) / 255.0)  # 0..1 weight

        # smooth noise field (so it looks organic, not pixel noise)
        noise = rng.random((img_h, img_w), dtype=np.float32)  # 0..1
        noise_img = Image.fromarray((noise * 255).astype(np.uint8), mode="L").filter(
            ImageFilter.GaussianBlur(radius=max(1, feather_px / 2))
        )
        noise = (np.asarray(noise_img, dtype=np.float32) / 255.0)  # 0..1

        # convert noise to centered range [-1, 1]
        noise = (noise * 2.0) - 1.0

        # apply noise only on the band: multiplicative perturbation
        # keeps center stable while breaking the straight border
        m2 = m * (1.0 + edge_noise_amp * band * noise)

        # clamp and rescale back to 0..255
        m2 = np.clip(m2, 0.0, 1.0)
        mask = Image.fromarray((m2 * 255.0).astype(np.uint8), mode="L")

    mask.save(out_path)
    return mask



In [18]:
mask = make_center_soft_rect_mask(
    img_w=2364,
    img_h=1282,
    fill_ratio=0.65,
    out_path="mask_improved_centered.png"
)


In [ ]:
from PIL import Image, ImageDraw

def make_center_rect_mask(img_w: int, img_h: int, fill_ratio: float = 0.75, out_path: str = "mask.png") -> Image.Image:
    """
    Create a centered rectangular mask of size (img_w, img_h).
    The rectangle covers `fill_ratio` of the width and height (e.g. 0.75 => 3/4).
    Returns the PIL mask (mode "L") and also saves it to out_path.
    """
    # clamp ratio to a safe range
    fill_ratio = max(0.0, min(1.0, float(fill_ratio)))

    mask = Image.new("L", (img_w, img_h), 0)
    draw = ImageDraw.Draw(mask)

    rect_w = int(round(img_w * fill_ratio))
    rect_h = int(round(img_h * fill_ratio))

    x0 = (img_w - rect_w) // 2
    y0 = (img_h - rect_h) // 2
    x1 = x0 + rect_w
    y1 = y0 + rect_h

    draw.rectangle([x0, y0, x1, y1], fill=200)

    mask.save(out_path)
    return mask


